In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from datetime import date

def getDataframe(jsonPath):
  with open(jsonPath) as f:
    data = json.load(f)

  #Create general data frame
  df = pd.DataFrame(columns=["user","likes", "followers", "item0", "item1", "date"])

  #Put the data in the data frame
  for i in range(0,len(data['output'])):
    df.loc[i] = [data['output'][i]["user"], data['output'][i]["likes"], data['output'][i]["followers"], data['output'][i]["item0"], data['output'][i]["item1"],data['output'][i]["date"]]

  #Eliminate rows which no value in likes from json
  df['likes'].replace('', np.nan, inplace=True)
  #print(df)
  df.dropna(subset=['likes'], inplace=True)
  #Se resetean los indices para el siguiente for
  df.reset_index(inplace=True, drop=True)
  
  #Change the type of followers and likes variables (String to int)
  df['followers']=df['followers'].astype(int)
  df['likes']=df['likes'].astype(int)
  df['date']=pd.to_datetime(df['date'])

  
  #Create new colum in panda series format in order to calculate difference in days
  today = pd.to_datetime("today")
  df['diff_days']= (today - df['date']).dt.days + 1
  df['f_score'] = 0.5

  # TODO: Define better formula
  # TODO: Change 36 to 9 and define better day mapping
  df['post_engagement'] = (df['likes'] / df['followers'])*100
  #* (37 - df['diff_days'])

  #INTERVAL PER POST
  #TODO Mirar como añadir bien el codigo extract_interval que saca media engagement de intervalos
  m0_5 = 2
  m500k_2M = 1
  df['value']=0.0
  for i in range(0, len(df['followers'])):
      if((df['followers'][i] > 0) and (df['followers'][i] < 7000)):
        #lo comparamos con la media del intervalo y ponderamos según nº followers, +followers +peso
        #TODO define average per intervals and weight 
        df['value'][i]=(df['post_engagement'][i] - m0_5)*0.1     
        '''el m0_5 será la media previamente extraida de ese intervalo'''

      elif ((df['followers'][i] >= 7000) and (df['followers'][i] < 20000000)):
          df['value'][i]=(df['post_engagement'][i] - m500k_2M)*0.5
    
  return df


def getLabelsDF(df,item):
  """ Parameters: Dataframe, name ofº the column that contains the item. Example 'item0' """
  df_item = pd.DataFrame(columns=["item","counts"])
  j=0
  for i in range(0, len(df[item].value_counts())):
    df_item.loc[j]=df[item].value_counts().index.tolist()[i],df[item].value_counts()[i]
    j=j+1

  return df_item



In [41]:
'''def engagementPost(df):
    # TODO: Define better formula
    # TODO: Change 36 to 9 and define better day mapping
    df['post_engagement'] = df['likes'] / df['followers'] 
    #* (37 - df['diff_days'])

    m0_5 = 2
    m500k_2M = 1
    df['value']=0.0
    for i in range(0, len(df['followers'])):
        if((df['followers'][i] > 0) and (df['followers'][i] < 7000)):
            #lo comparamos con la media del intervalo y ponderamos según nº followers, +followers +peso
            #TODO define average per intervals and weight 
            df['value'][i]=(df['post_engagement'][i] - m0_5)*0.1 

        elif ((df['followers'][i] >= 7000) and (df['followers'][i] < 20000000)):
            df['value'][i]=(df['post_engagement'][i] - m500k_2M)*0.5
    return df'''

In [2]:
if __name__ == "__main__":
  postsDF = getDataframe("recognition-output.json")
  topLabelsDF = getLabelsDF(postsDF,'item0')
  bottomLabelsDF = getLabelsDF(postsDF,'item1')

C:\Users\joser\AppData\Local\Temp/ipykernel_18856/510737981.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['value'][i]=(df['post_engagement'][i] - m500k_2M)*0.5
C:\Users\joser\AppData\Local\Temp/ipykernel_18856/510737981.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['value'][i]=(df['post_engagement'][i] - m0_5)*0.1


In [63]:
postsDF
#topLabelsDF
#bottomLabelsDF

,user,likes,followers,item0,item1,date,diff_days,f_score,post_engagement,value
0,raquelreitx,23158,514246,N/A N/A,N/A N/A,2021-02-11,282,0.5,4.503292,1.751646
1,mariapombo,56418,2236175,N/A N/A,N/A N/A,2021-03-11,254,0.5,2.522969,0.761484
2,mariapombo,39770,2236175,N/A N/A,N/A N/A,2021-10-31,20,0.5,1.778483,0.389242
3,mariapombo,50204,2236175,jersey gris,pantalones gris,2021-10-30,21,0.5,2.245084,0.622542
4,mariapombo,52838,2236175,N/A N/A,N/A N/A,2021-10-29,22,0.5,2.362874,0.681437
...,...,...,...,...,...,...,...,...,...,...
280,alessandro.piacenti,1574,108459,abrigo_manga_larga marron,N/A N/A,2021-04-11,223,0.5,1.451240,0.225620
281,chadhaupt_,958,41739,jersey blanco,pantalones gris,2021-04-11,223,0.5,2.295216,0.647608
282,chadhaupt_,858,41739,abrigo_manga_larga gris,pantalones gris,2021-01-11,313,0.5,2.055631,0.527816
283,chadhaupt_,757,41739,camiseta_manga_corta negro,pantalones negro,2021-10-29,22,0.5,1.813652,0.406826
